In [ ]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np

from pykern.pkcollections import PKDict
from pykern import pkdebug

import rslaser.rscavity.laser_cavity as rslc
import rslaser.rsoptics.element as rse
import rslaser.utils.constants as rsc

import scipy.constants as const
import time

In [ ]:
# Define parameters for laser pulse starting at center of crystal
a0 = 0.01             # 0.85e-9 * lambda [microns] * Sqrt(Intensity [W/cm^2])
lambda0 = 8.e-7       # wavelength [m]
tau_fwhm = 0.1 / const.c / math.sqrt(2.)   # FWHM pulse length [s]
z_waist = 0.          # longitudinal waist position [m]
z_center = 0.0        # longitudinal position of pulse center [m]

# deviations
d_lambda = 0.         # full wavelength chirp across laser pulse [m]
dw0x = 0.0            # deviation of horizontal waist from w0
dw0y = 0.0            # deviation of vertical waist from w0
dzwx = 0.0            # deviation of horizontal waist position from z_waist
dzwy = 0.0            # deviation of vertical waist position from z_waist
x_shift = 0.          # bulk horizontal shift of pulse position
y_shift = 0.          # bulk vertical shift of pulse position

# numerical values
num_slices = 5        # desired number of slices (i.e. SRW wavefronts) to represent the pulse

# other DERIVED quantities
d_to_w = z_waist - z_center   # so-called 'drift to waist' (required by SRW)

# Cavity propagation notebook

This notebook propagates a wavefront through an optical cavity. It demonstrates the use of the cavity.py 
code.
The initial wavefront is a Hermite-Gaussian mode located at the center of a cavity
of length L_cav. There is a crystal located at the center of the cavity of length L_cryst. 
The wavefront is propagated through the crystal and then to the end of the cavity.

In [ ]:
k=PKDict(
# Define Cavity length
L_cav = 1, #Length of cavity [m]

# Define right and left mirror focal lengths
df = 0.3, #Focal length difference from confocal case [m]
)

# Define parameters for laser pulse starting at center of crystal
k.a0 = a0
k.lambda0 = lambda0
k.tau_fwhm = tau_fwhm
k.z_center = z_center
k.z_waist = z_waist
k.d_to_w = d_to_w

k.d_lambda = d_lambda
k.dw0x = dw0x
k.dw0y = dw0y
k.dzwx = dzwx
k.dzwy = dzwy
k.x_shift = x_shift
k.y_shift = y_shift

k.nslice = num_slices

k.dfR = k.df
k.dfL = k.df
k.f=k.L_cav/4+k.df #focal length

k.lens_left_focal_length = k.f
k.lens_right_focal_length = k.f

#Define Crystal parameters
k.L_cryst = 0.1 #Length of crystal [m]
k.n0=1.75 #index of refraction on axis
k.n2=0.0 #radial variation of index of refraction: n(r) = n0 - 0.5 n2 r^2
k.drift_right_length=k.L_cav/2-k.L_cryst/2
k.drift_left_length=k.drift_right_length

k.L_eff = k.L_cav+(1/k.n0 - 1)*k.L_cryst #Define effective length as path length reduced by index of refraction n0
print("L_eff = %f m" % k.L_eff)
k.beta0 = np.sqrt(k.L_eff*k.f-k.L_eff**2/4)
print("beta0 = %f m" % k.beta0)
k.sigx0 = np.sqrt(k.lambda0*k.beta0/4/np.pi)
print("sigx0 = %f m" % k.sigx0)
k.sigrW = k.sigx0
k.w0 = k.sigx0 * math.sqrt(2.)

k.L_half_cryst=k.L_cryst/2

In [ ]:
#create laser cavity
lc = rslc.LaserCavity(k)

#nslice=nslice,L_half_cryst=L_cryst/2, 
#                        drift_right_length=drift_right_length,drift_left_length=drift_left_length, 
#                        lens_left_focal_length=lens_left_focal_length, 
#                        lens_right_focal_length=lens_right_focal_length,n0=n0,n2=n2,sigrW=sigx0

#output initial parameters
svals = lc.laser_pulse.pulsePos()
(lpsxvals,lpsyvals) = lc.laser_pulse.rmsvals()
ivals = lc.laser_pulse.intensity_vals()
evals = lc.laser_pulse.energyvals()

In [ ]:
#Plot rms values along laser pulse
plt.plot(svals,lpsyvals)
plt.xlabel('s [m]')
plt.ylabel('rms x [m]')
plt.title('RMS Beam size along laser pulse')
plt.show()

#Plot pulse intensity along laser pulse
plt.plot(svals,ivals)
plt.ylabel('pulse intensity []')
plt.xlabel('s [m]')
plt.show()

plt.title('Slice energy along laser pulse')
plt.plot(svals,evals)
plt.xlabel('s [m]')
plt.ylabel('Energy [ev]')
plt.show()

In [ ]:
start_time = time.time()
(svals, sxvals, syvals) = lc.propagate(num_cycles=4)

fig, ax = plt.subplots()
ax.plot(svals, sxvals)
print("Simulation time: %s seconds" %(round((time.time() - start_time),5)))

In [ ]:
#get final laser pulse rms values and intensity values along pulse
svals = lc.laser_pulse.pulsePos()
(lpsxvals,lpsyvals) = lc.laser_pulse.rmsvals()
ivals = lc.laser_pulse.intensity_vals()
evals = lc.laser_pulse.energyvals()

In [ ]:
#Plots
plt.title('RMS Beam size along laser pulse')
plt.plot(svals,lpsyvals)
plt.xlabel('s [m]')
plt.ylabel('rms x [m]')
plt.show()

plt.title('Intensity along laser pulse')
plt.plot(svals,ivals)
plt.ylabel('pulse intensity []')
plt.xlabel('s [m]')
plt.show()

plt.title('Slice energy along laser pulse')
plt.plot(svals,evals)
plt.xlabel('s [m]')
plt.ylabel('Energy [ev]')
plt.show()